In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import mplhep
mplhep.style.use("CMS")

import sys
sys.path += ["../../mlpf/"]
sys.path += ["../../mlpf/plotting/"]
from plot_utils import cms_label

In [ ]:
s_fused_bs2 = """
timing/gpu_fp32_fused_bs2.txt:Nelem=2560 mean_time=20.38 ms stddev_time=0.08 ms mem_used=1268 MB
timing/gpu_fp32_fused_bs2.txt:Nelem=5120 mean_time=49.49 ms stddev_time=0.06 ms mem_used=1537 MB
timing/gpu_fp32_fused_bs2.txt:Nelem=10240 mean_time=135.40 ms stddev_time=0.07 ms mem_used=2074 MB
"""

s_fused_bs4 = """
timing/gpu_fp32_fused_bs4.txt:Nelem=2560 mean_time=19.93 ms stddev_time=0.03 ms mem_used=1537 MB
timing/gpu_fp32_fused_bs4.txt:Nelem=5120 mean_time=48.39 ms stddev_time=0.04 ms mem_used=2074 MB
timing/gpu_fp32_fused_bs4.txt:Nelem=10240 mean_time=134.42 ms stddev_time=0.06 ms mem_used=3147 MB
"""

s_fused_bs1 = """
timing/gpu_fp32_fused.txt:Nelem=2560 mean_time=18.99 ms stddev_time=0.10 ms mem_used=1134 MB
timing/gpu_fp32_fused.txt:Nelem=5120 mean_time=50.43 ms stddev_time=0.10 ms mem_used=1268 MB
timing/gpu_fp32_fused.txt:Nelem=10240 mean_time=137.60 ms stddev_time=0.14 ms mem_used=1537 MB
"""

s_unfused = """
timing/gpu_fp32_unfused.txt:Nelem=2560 mean_time=79.48 ms stddev_time=0.35 ms mem_used=2208 MB
timing/gpu_fp32_unfused.txt:Nelem=5120 mean_time=289.87 ms stddev_time=1.72 ms mem_used=6503 MB
"""

In [ ]:
def parse_str(s):
    data = pd.DataFrame()
    Nelem = []
    mean_time = []
    stddev_time = []
    mem_used = []
    for line in s.strip().split("\n"):
        elems = line.split(":")[1].split()
        print(elems)
        Nelem.append(int(elems[0].split("=")[1]))
        mean_time.append(float(elems[1].split("=")[1]))
        stddev_time.append(float(elems[3].split("=")[1]))
        mem_used.append(float(elems[5].split("=")[1]))
    data["Nelem"] = Nelem
    data["mean_time"] = mean_time
    data["stddev_time"] = stddev_time
    data["mem_used"] = mem_used
    return data

In [ ]:
data_fused_bs1 = parse_str(s_fused_bs1)
data_fused_bs2 = parse_str(s_fused_bs2)
data_fused_bs4 = parse_str(s_fused_bs4)
data_unfused = parse_str(s_unfused)

In [ ]:
plt.figure()
ax = plt.axes()
plt.errorbar(data_unfused["Nelem"], data_unfused["mean_time"], yerr=data_unfused["stddev_time"], marker="o", label="unfused")
plt.errorbar(data_fused_bs1["Nelem"], data_fused_bs1["mean_time"], yerr=data_fused_bs1["stddev_time"], marker="^", label="fused, bs1")
plt.errorbar(data_fused_bs4["Nelem"], data_fused_bs4["mean_time"], yerr=data_fused_bs4["stddev_time"], marker="v", label="fused, bs4")
plt.xticks(data_fused_bs1["Nelem"])
plt.ylabel("Runtime per event [ms]")
plt.xlabel("Tracks/clusters per event")
plt.legend(loc=1, title="MLPF-2x3x256\nONNX backend\nA100-1g.10gb")
plt.ylim(0,400)
cms_label(ax)
plt.savefig("runtime.pdf")

In [ ]:
plt.figure()
ax = plt.axes()
plt.errorbar(data_unfused["Nelem"], data_unfused["mem_used"], marker="o", label="unfused")
plt.errorbar(data_fused_bs1["Nelem"], data_fused_bs1["mem_used"], marker="^", label="fused, bs1")
plt.errorbar(data_fused_bs4["Nelem"], data_fused_bs4["mem_used"]/4, marker="v", label="fused, bs4")
plt.xticks(data_fused_bs1["Nelem"])
plt.ylabel("GPU memory per event [MB]")
plt.xlabel("Tracks/clusters per event")
plt.axhline(10200, color="red", ls="--", label="GPU limit")
#plt.yscale("log")
#ytick = [256,512,1024,2048,4096,8192]
#plt.yticks(ytick, ytick)
plt.legend(loc=1, title="MLPF-2x3x256\nONNX backend\nA100-1g.10gb")
cms_label(ax)
plt.savefig("memory.pdf")

In [ ]:
s_cpu_pf = """
log_cpu_pf.txt:TimeModule> 35002 1 particleFlowTmp PFProducer 0.00893436
log_cpu_pf.txt:TimeModule> 35005 1 particleFlowTmp PFProducer 0.00696006
log_cpu_pf.txt:TimeModule> 35001 1 particleFlowTmp PFProducer 0.0205714
log_cpu_pf.txt:TimeModule> 35004 1 particleFlowTmp PFProducer 0.0115013
log_cpu_pf.txt:TimeModule> 35003 1 particleFlowTmp PFProducer 0.010012
log_cpu_pf.txt:TimeModule> 35006 1 particleFlowTmp PFProducer 0.00605446
log_cpu_pf.txt:TimeModule> 35010 1 particleFlowTmp PFProducer 0.0122532
log_cpu_pf.txt:TimeModule> 35009 1 particleFlowTmp PFProducer 0.0221017
log_cpu_pf.txt:TimeModule> 35008 1 particleFlowTmp PFProducer 0.00843328
log_cpu_pf.txt:TimeModule> 35011 1 particleFlowTmp PFProducer 0.0095517
log_cpu_pf.txt:TimeModule> 35012 1 particleFlowTmp PFProducer 0.00850458
log_cpu_pf.txt:TimeModule> 35014 1 particleFlowTmp PFProducer 0.0196761
log_cpu_pf.txt:TimeModule> 35007 1 particleFlowTmp PFProducer 0.00726191
log_cpu_pf.txt:TimeModule> 35017 1 particleFlowTmp PFProducer 0.0126049
log_cpu_pf.txt:TimeModule> 35018 1 particleFlowTmp PFProducer 0.00476037
log_cpu_pf.txt:TimeModule> 35013 1 particleFlowTmp PFProducer 0.0111422
log_cpu_pf.txt:TimeModule> 35016 1 particleFlowTmp PFProducer 0.0135155
log_cpu_pf.txt:TimeModule> 35019 1 particleFlowTmp PFProducer 0.00631518
log_cpu_pf.txt:TimeModule> 35015 1 particleFlowTmp PFProducer 0.00879818
log_cpu_pf.txt:TimeModule> 35021 1 particleFlowTmp PFProducer 0.0111998
"""

s_cpu_mlpf = """
log_cpu.txt:TimeModule> 35002 1 particleFlowTmp MLPFProducer 9.4116
log_cpu.txt:TimeModule> 35005 1 particleFlowTmp MLPFProducer 8.02389
log_cpu.txt:TimeModule> 35001 1 particleFlowTmp MLPFProducer 13.4437
log_cpu.txt:TimeModule> 35004 1 particleFlowTmp MLPFProducer 10.4151
log_cpu.txt:TimeModule> 35003 1 particleFlowTmp MLPFProducer 12.1385
log_cpu.txt:TimeModule> 35006 1 particleFlowTmp MLPFProducer 7.06085
log_cpu.txt:TimeModule> 35010 1 particleFlowTmp MLPFProducer 12.1508
log_cpu.txt:TimeModule> 35009 1 particleFlowTmp MLPFProducer 13.2121
log_cpu.txt:TimeModule> 35008 1 particleFlowTmp MLPFProducer 10.3394
log_cpu.txt:TimeModule> 35011 1 particleFlowTmp MLPFProducer 9.24309
log_cpu.txt:TimeModule> 35012 1 particleFlowTmp MLPFProducer 9.26367
log_cpu.txt:TimeModule> 35014 1 particleFlowTmp MLPFProducer 13.2224
log_cpu.txt:TimeModule> 35007 1 particleFlowTmp MLPFProducer 8.03034
log_cpu.txt:TimeModule> 35017 1 particleFlowTmp MLPFProducer 12.1319
log_cpu.txt:TimeModule> 35018 1 particleFlowTmp MLPFProducer 5.83649
log_cpu.txt:TimeModule> 35013 1 particleFlowTmp MLPFProducer 11.9684
log_cpu.txt:TimeModule> 35016 1 particleFlowTmp MLPFProducer 10.2273
log_cpu.txt:TimeModule> 35019 1 particleFlowTmp MLPFProducer 6.9992
log_cpu.txt:TimeModule> 35015 1 particleFlowTmp MLPFProducer 7.96592
log_cpu.txt:TimeModule> 35021 1 particleFlowTmp MLPFProducer 11.9789
"""

s_gpu_mlpf = """
log_gpu.txt:TimeModule> 35002 1 particleFlowTmp MLPFProducer 0.177305
log_gpu.txt:TimeModule> 35005 1 particleFlowTmp MLPFProducer 0.0156437
log_gpu.txt:TimeModule> 35001 1 particleFlowTmp MLPFProducer 0.0187983
log_gpu.txt:TimeModule> 35004 1 particleFlowTmp MLPFProducer 0.0158696
log_gpu.txt:TimeModule> 35003 1 particleFlowTmp MLPFProducer 0.0171756
log_gpu.txt:TimeModule> 35006 1 particleFlowTmp MLPFProducer 0.0125436
log_gpu.txt:TimeModule> 35010 1 particleFlowTmp MLPFProducer 0.0167758
log_gpu.txt:TimeModule> 35009 1 particleFlowTmp MLPFProducer 0.0184546
log_gpu.txt:TimeModule> 35008 1 particleFlowTmp MLPFProducer 0.0161449
log_gpu.txt:TimeModule> 35011 1 particleFlowTmp MLPFProducer 0.0146528
log_gpu.txt:TimeModule> 35012 1 particleFlowTmp MLPFProducer 0.0149266
log_gpu.txt:TimeModule> 35014 1 particleFlowTmp MLPFProducer 0.0183499
log_gpu.txt:TimeModule> 35007 1 particleFlowTmp MLPFProducer 0.0130879
log_gpu.txt:TimeModule> 35017 1 particleFlowTmp MLPFProducer 0.0170359
log_gpu.txt:TimeModule> 35018 1 particleFlowTmp MLPFProducer 0.0111724
log_gpu.txt:TimeModule> 35013 1 particleFlowTmp MLPFProducer 0.0167873
log_gpu.txt:TimeModule> 35016 1 particleFlowTmp MLPFProducer 0.0162624
log_gpu.txt:TimeModule> 35019 1 particleFlowTmp MLPFProducer 0.0118865
log_gpu.txt:TimeModule> 35015 1 particleFlowTmp MLPFProducer 0.0126998
log_gpu.txt:TimeModule> 35021 1 particleFlowTmp MLPFProducer 0.0169669
"""

In [ ]:
def parse_cmssw(s):
    s = s.strip()
    times = []
    for line in s.split("\n"):
        if len(line)>0:
            time = float(line.split()[-1])*1000.0
            times.append(time)
    return np.array(times)
    

In [ ]:
times_pf_cpu = parse_cmssw(s_cpu_pf)
times_mlpf_cpu = parse_cmssw(s_cpu_mlpf)
times_mlpf_gpu = parse_cmssw(s_gpu_mlpf)

In [ ]:
plt.bar([0,1], [np.mean(times_pf_cpu[1:]), np.mean(times_mlpf_gpu[1:])], yerr=[np.std(times_pf_cpu[1:]), np.std(times_mlpf_gpu[1:])])
plt.xticks([0,1], ["PF on CPU", "MLPF-2x6x512-relu on GPU"])
plt.ylabel("runtime per event [ms]")